In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame({'A': np.random.randint(0, 10, 10)})
df

,A
0,5
1,6
2,8
3,4
4,7
5,3
6,2
7,7
8,7
9,2


In [3]:
df['B'] = df['A'].diff()

In [4]:
df

,A,B
0,5,NaN
1,6,1.0
2,8,2.0
3,4,-4.0
4,7,3.0
5,3,-4.0
6,2,-1.0
7,7,5.0
8,7,0.0
9,2,-5.0


In [5]:
x, x_diff = df['A'].iloc[0], df['B'].iloc[1:]

In [6]:
np.r_[x, x_diff]

array([ 5.,  1.,  2., -4.,  3., -4., -1.,  5.,  0., -5.])

In [7]:
x

5

In [8]:
x_diff

1    1.0
2    2.0
3   -4.0
4    3.0
5   -4.0
6   -1.0
7    5.0
8    0.0
9   -5.0
Name: B, dtype: float64

In [9]:
# .r_ concatenates x= first obervation to the beginning of the x_diff column
np.r_[x, x_diff].cumsum().astype(int)

array([5, 6, 8, 4, 7, 3, 2, 7, 7, 2])

In [10]:
df['C'] = np.r_[x, x_diff].cumsum().astype(int)

In [11]:
df

,A,B,C
0,5,NaN,5
1,6,1.0,6
2,8,2.0,8
3,4,-4.0,4
4,7,3.0,7
5,3,-4.0,3
6,2,-1.0,2
7,7,5.0,7
8,7,0.0,7
9,2,-5.0,2


In [12]:
window_setting = 4
df['moving_avg'] = df['A'].rolling(window=window_setting).mean().shift()
df['moving_avg_diff'] = df['A'] - df['A'].rolling(window=window_setting).mean().shift()
df

,A,B,C,moving_avg,moving_avg_diff
0,5,NaN,5,NaN,NaN
1,6,1.0,6,NaN,NaN
2,8,2.0,8,NaN,NaN
3,4,-4.0,4,NaN,NaN
4,7,3.0,7,5.75,1.25
5,3,-4.0,3,6.25,-3.25
6,2,-1.0,2,5.50,-3.50
7,7,5.0,7,4.00,3.00
8,7,0.0,7,4.75,2.25
9,2,-5.0,2,4.75,-2.75


## Rescaling from Moving Average


In [13]:

test_set = df.iloc[window_setting:, :]
train_set = df.iloc[:window_setting]

original_tail = train_set['A'].tail(window_setting)

for key, item in test_set['moving_avg_diff'].items():
    rolling_avg = original_tail.tail(window_setting).mean()
    unscaled_result = item + rolling_avg
    original_tail = original_tail.append(pd.Series([unscaled_result]))

df['unscaled_y'] = list(original_tail)
df

,A,B,C,moving_avg,moving_avg_diff,unscaled_y
0,5,NaN,5,NaN,NaN,5.0
1,6,1.0,6,NaN,NaN,6.0
2,8,2.0,8,NaN,NaN,8.0
3,4,-4.0,4,NaN,NaN,4.0
4,7,3.0,7,5.75,1.25,7.0
5,3,-4.0,3,6.25,-3.25,3.0
6,2,-1.0,2,5.50,-3.50,2.0
7,7,5.0,7,4.00,3.00,7.0
8,7,0.0,7,4.75,2.25,7.0
9,2,-5.0,2,4.75,-2.75,2.0


In [14]:
test_set = df.iloc[window_setting:, :]
train_set = df.iloc[:window_setting]

original_tail = train_set['A'].tail(window_setting)

for key, item in test_set['moving_avg_diff'].items():
    rolling_avg = original_tail.tail(window_setting).mean()
    unscaled_result = item + rolling_avg
    original_tail = original_tail.append(pd.Series([unscaled_result]))

df['unscaled_y'] = list(original_tail)
df

,A,B,C,moving_avg,moving_avg_diff,unscaled_y
0,5,NaN,5,NaN,NaN,5.0
1,6,1.0,6,NaN,NaN,6.0
2,8,2.0,8,NaN,NaN,8.0
3,4,-4.0,4,NaN,NaN,4.0
4,7,3.0,7,5.75,1.25,7.0
5,3,-4.0,3,6.25,-3.25,3.0
6,2,-1.0,2,5.50,-3.50,2.0
7,7,5.0,7,4.00,3.00,7.0
8,7,0.0,7,4.75,2.25,7.0
9,2,-5.0,2,4.75,-2.75,2.0


In [15]:
window_setting = 4
df['exp_moving_avg'] = df['A'].ewm(span=4, min_periods=4, adjust=False).mean().shift()
df['ewm_diff'] = df['A'] - df['exp_moving_avg']
df

,A,B,C,moving_avg,moving_avg_diff,unscaled_y,exp_moving_avg,ewm_diff
0,5,NaN,5,NaN,NaN,5.0,NaN,NaN
1,6,1.0,6,NaN,NaN,6.0,NaN,NaN
2,8,2.0,8,NaN,NaN,8.0,NaN,NaN
3,4,-4.0,4,NaN,NaN,4.0,NaN,NaN
4,7,3.0,7,5.75,1.25,7.0,5.464000,1.536000
5,3,-4.0,3,6.25,-3.25,3.0,6.078400,-3.078400
6,2,-1.0,2,5.50,-3.50,2.0,4.847040,-2.847040
7,7,5.0,7,4.00,3.00,7.0,3.708224,3.291776
8,7,0.0,7,4.75,2.25,7.0,5.024934,1.975066
9,2,-5.0,2,4.75,-2.75,2.0,5.814961,-3.814961


In [17]:
original_tail = train_set['A'].tail(window_setting)

#print(original_tail.tail(4).ewm(span=4, adjust=False).mean()[3])

rolling_mean_add = original_tail.tail(4).ewm(span=4, adjust=False).mean()[3]

unscaled = test_set.iloc[0,:] + rolling_mean_add
original_tail = original_tail.append(pd.Series([float(unscaled)]))
original_tail
#print(original_tail.tail(4).ewm(span=4, adjust=False).mean())


TypeError: cannot convert the series to <class 'float'>

In [101]:
# Unscaled values (tail) to calculate moving average from
print(original_tail.tail(4))

result = original_tail.tail(4).ewm(span=4, adjust=False).mean()
result = result.reset_index().iloc[:,1]

print(result)
print()
print('Exponential Weighted Mean of most recent 4 unscaled values')
print(result[3])

1    0.0
2    2.0
3    5.0
0    1.0
dtype: float64
0    0.000
1    0.800
2    2.480
3    1.888
Name: 0, dtype: float64

Exponential Weighted Mean of most recent 4 unscaled values
1.888


#### Rescaling Back to Scale

In [103]:
test_set = df.iloc[window_setting:, :]
train_set = df.iloc[:window_setting]

original_tail = train_set['A'].tail(window_setting)

for key, item in test_set['ewm_diff'].items():
    ewm_rolling_avg = original_tail.tail(4).ewm(span=4, adjust=False).mean()
    ewm_rolling_avg = ewm_rolling_avg.reset_index().iloc[:,1]
    ewm_rolling_avg = ewm_rolling_avg[3]
    
    # original_tail.tail(4).ewm(span=4, adjust=False).mean()[window_setting-1]
    #print(original_tail.tail(4))
    print('rolling avg', ewm_rolling_avg)
    #print(item)
    #print('sum', item + ewm_rolling_avg)
    
    unscaled_result = item + ewm_rolling_avg
    original_tail = original_tail.append(pd.Series([unscaled_result]))

df['unscaled_y'] = list(original_tail)
df

rolling avg 3.992
rolling avg 1.888
rolling avg 1.8291199999999999
rolling avg 3.2270719999999997
rolling avg 3.3141632000000003
rolling avg 5.2894848


,A,B,C,moving_avg,moving_avg_diff,unscaled_y,exp_moving_avg,ewm_diff
0,7,NaN,7,NaN,NaN,7.000000,NaN,NaN
1,0,-7.0,0,NaN,NaN,0.000000,NaN,NaN
2,2,2.0,2,NaN,NaN,2.000000,NaN,NaN
3,5,3.0,5,NaN,NaN,5.000000,NaN,NaN
4,1,-4.0,1,3.50,-2.50,1.000000,3.992000,-2.992000
5,2,1.0,2,2.00,0.00,1.092800,2.795200,-0.795200
6,5,3.0,5,2.50,2.50,4.352000,2.477120,2.522880
7,5,0.0,5,3.25,1.75,4.740800,3.486272,1.513728
8,9,4.0,9,3.25,5.75,8.222400,4.091763,4.908237
9,2,-7.0,2,5.25,-3.25,1.234427,6.055058,-4.055058


In [16]:
test_set = test_set['ewm_diff'].reset_index().iloc[:, 1:]

KeyError: 'ewm_diff'